### 보스턴 주택 가격 데이터 세트를 Peceptron 기반에서 학습 및 테스트하기 위한 데이터 로드
* 보스턴 주택 가격 데이터 csv파일을 다운로드하고 이를 DataFrame으로 생성

In [ ]:
!wget -O boston_price.csv https://raw.githubusercontent.com/chulminkw/CNN_PG_Torch/refs/heads/main/data/boston_house_price.csv

In [ ]:
import pandas as pd
import numpy as np

boston_df = pd.read_csv('./boston_price.csv')
boston_df.head()

In [ ]:
print(boston_df.shape)

### Weight와 Bias의 Update 값을 계산하는 함수 생성.
* w1은 RM(방의 계수) 피처의 Weight 값
* w2는 LSTAT(하위계층 비율) 피처의 Weight 값
* bias는 Bias
* N은 입력 데이터 건수
![](https://raw.githubusercontent.com/chulminkw/CNN_PG_Torch/main/image/gradient_descent.png)


In [ ]:
import torch

# gradient_descent()함수에서 반복적으로 호출되면서 update될 weight/bias 값을 계산하는 함수. 
# rm은 RM(방 개수), lstat(하위계층 비율), target은 PRICE임. 전체 해당 tensor가 다 입력됨. 
# 반환 값은 weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 loss로 반환.
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate=0.01):
    # 데이터 건수
    N = target.shape[0]
    # 예측 값. 
    predicted = w1 * rm + w2 * lstat + bias
    # 실제값과 예측값의 차이
    diff = target - predicted 
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N) * learning_rate * (torch.matmul(rm, diff))
    w2_update = -(2/N) * learning_rate * (torch.matmul(lstat, diff))
    bias_update = -(2/N) * learning_rate * torch.sum(diff)
    
    # Mean Squared Error값을 계산. 
    mse_loss = torch.mean(diff ** 2)
    
    # weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 반환. 
    return bias_update, w1_update, w2_update, mse_loss

### Gradient Descent 를 적용하는 함수 생성
* iter_epochs 수만큼 반복적으로 get_update_weights_value()를 호출하여 update될 weight/bias값을 구한 뒤 Weight/Bias를 Update적용. 

In [ ]:
# RM, LSTAT feature tensor와 PRICE target tensor를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용. 
def gradient_descent(features, target, iter_epochs=1000, learning_rate=0.01, verbose=True):
    # w1, w2는 1차원 tensor로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 tensor로 변환하되 초기 값은 1로 설정. 
    w1 = torch.zeros(1, dtype=torch.float32)
    w2 = torch.zeros(1, dtype=torch.float32)
    bias = torch.ones(1, dtype=torch.float32)
    print('최초 w1, w2, bias:', w1.item(), w2.item(), bias.item())
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 tensor형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(1, iter_epochs+1):
        # weight/bias update 값 계산
        bias_update, w1_update, w2_update, loss = get_update_weights_value(bias, w1, w2, 
                                                                           rm, lstat, target, learning_rate=0.01)
        
        # weight/bias의 update 적용.
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose: # 10회 epochs 시마다 출력
            if i % 10 == 0:
                print(f'Epoch: {i}/{iter_epochs}')
                print(f'w1: {w1.item()}, w2: {w2.item()}, bias: {bias.item()}, loss: {loss.item()}')
        
    return w1, w2, bias

### Gradient Descent 적용
* 신경망은 데이터를 정규화/표준화 작업을 미리 선행해 주어야 함. 
* 이를 위해 사이킷런의 MinMaxScaler를 이용하여 개별 feature값은 0~1사이 값으로 변환후 학습 적용.

In [ ]:
import torch
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features_np = scaler.fit_transform(boston_df[['RM', 'LSTAT']])

scaled_features_ts = torch.from_numpy(scaled_features_np)
targets_ts = torch.from_numpy(boston_df['PRICE'].values)

w1, w2, bias = gradient_descent(scaled_features_ts, targets_ts, iter_epochs=5000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1.item(), w2.item(), bias.item())

### 계산된 Weight와 Bias를 이용하여 Price 예측
* 예측 feature 역시 0~1사이의 scaled값을 이용하고 Weight와 bias를 적용하여 예측값 계산. 

In [ ]:
predicted = scaled_features_ts[:, 0]*w1 + scaled_features_ts[:, 1]*w2 + bias
boston_df['PREDICTED_PRICE'] = predicted.cpu().numpy()
boston_df.head(10)

In [ ]:
from sklearn.metrics import mean_squared_error

total_mse = mean_squared_error(boston_df['PRICE'], boston_df['PREDICTED_PRICE'])
print(total_mse)

In [ ]:
from sklearn.model_selection import train_test_split

tr_features, test_features, tr_target, test_target = train_test_split(scaled_features_np, boston_df['PRICE'].values, 
                                                                      test_size=0.3, random_state=2025)
print(tr_features.shape, tr_target.shape, test_features.shape, test_target.shape)

In [ ]:
tr_features_ts = torch.from_numpy(tr_features)
tr_targets_ts = torch.from_numpy(tr_target)

w1, w2, bias = gradient_descent(tr_features_ts, tr_targets_ts, iter_epochs=5000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1.item(), w2.item(), bias.item())

In [ ]:
test_features_ts = torch.from_numpy(test_features)
test_predicted_ts = test_features_ts[:, 0]*w1 + test_features_ts[:, 1]*w2 + bias

boston_test_df = pd.DataFrame({
    'RM': test_features[:, 0],
    'LSTAT': test_features[:, 1],
    'PRICE': test_target,
    'PREDICTED_PRICE': test_predicted_ts.cpu().numpy()
})

boston_test_df.head(20)

In [ ]:
test_total_mse = mean_squared_error(boston_test_df['PRICE'], boston_test_df['PREDICTED_PRICE'])
print(test_total_mse)

### Stochastic Gradient Descent와 Mini Batch Gradient Descent 구현
* SGD 는 전체 데이터에서 한건만 임의로 선택하여 Gradient Descent 로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용
* Mini Batch GD는 전체 데이터에서 Batch 건수만큼 데이터를 선택하여 Gradient Descent로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용

### SGD 기반으로 Weight/Bias update 값 구하기

In [ ]:
import torch

# get_update_weights_value() 함수와 거의 유사. 
# 인자로 들어오는 rm_sgd, lstat_sgd, target_sgd은 단 1개의 원소를 가지는 tensor임. 
def get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate=0.01):
    # 데이터 건수
    N = target_sgd.shape[0]
    # 예측 값. 
    predicted_sgd = w1 * rm_sgd + w2 * lstat_sgd + bias
    # 실제값과 예측값의 차이
    diff_sgd = target_sgd - predicted_sgd 
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N) * learning_rate * (torch.matmul(rm_sgd, diff_sgd))
    w2_update = -(2/N) * learning_rate * (torch.matmul(lstat_sgd, diff_sgd))
    bias_update = -(2/N) * learning_rate * torch.sum(diff_sgd)
    
    # weight와 bias가 update되어야 할 값 반환. 
    return bias_update, w1_update, w2_update

### SGD 수행하기

In [ ]:
# RM, LSTAT feature tensor와 PRICE target tensor를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용. 
def st_gradient_descent(features, target, iter_epochs=1000, learning_rate=0.01, verbose=True):
    # random seed 값 설정. 
    torch.manual_seed(2025)
    # w1, w2는 1차원 tensor로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 tensor로 변환하되 초기 값은 1로 설정.
    w1 = torch.zeros(1, dtype=torch.float32)
    w2 = torch.zeros(1, dtype=torch.float32)
    bias = torch.ones(1, dtype=torch.float32)
    print('최초 w1, w2, bias:', w1.item(), w2.item(), bias.item())
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 tensor형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(1, iter_epochs+1):
        # iteration 시마다 stochastic gradient descent 를 수행할 데이터를 한개만 추출. 
        #추출할 데이터의 인덱스를  로 선택. 
        stochastic_index = torch.randint(0, target.shape[0], size=(1,))
        rm_sgd = rm[stochastic_index]
        lstat_sgd = lstat[stochastic_index]
        target_sgd = target[stochastic_index]
        # weight/bias update 값 계산. loss 반환 없음. 
        bias_update, w1_update, w2_update = get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, 
                                                                     target_sgd, learning_rate=0.01)
        # weight/bias의 update 적용.
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose: # 100회 iteration 시마다 출력
            if i % 100 == 0:
                print(f'Epoch: {i}/{iter_epochs}')
                # Loss는 전체 학습 데이터 기반으로 구해야 함. 아래는 전체 학습 feature 기반의 예측 및 loss임.  
                predicted = w1 * rm + w2*lstat + bias
                diff = target - predicted
                loss = torch.mean(diff ** 2)
                print(f'w1: {w1.item()}, w2: {w2.item()}, bias: {bias.item()}, loss: {loss.item()}')
        
    return w1, w2, bias

In [ ]:
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 학습과 테스트용 feature와 target 분리. 
def get_scaled_train_test_feature_target_ts(data_df):
    # RM, LSTAT Feature에 Scaling 적용
    scaler = MinMaxScaler()
    scaled_features_np = scaler.fit_transform(data_df[['RM', 'LSTAT']])
    # 학습 feature, 테스트 feature, 학습 target, test_target으로 분리. 
    tr_features, test_features, tr_target, test_target = train_test_split(scaled_features_np, 
                                                                          data_df['PRICE'].values, 
                                                                          test_size=0.3, random_state=2025)
    # 학습 feature와 target을 tensor로 변환. 
    tr_ftr_ts = torch.from_numpy(tr_features)
    tr_tgt_ts = torch.from_numpy(tr_target)
    test_ftr_ts = torch.from_numpy(test_features)
    test_tgt_ts = torch.from_numpy(test_target)
    
    return tr_ftr_ts, tr_tgt_ts, test_ftr_ts, test_tgt_ts

tr_ftr_ts, tr_tgt_ts, test_ftr_ts, test_tgt_ts = get_scaled_train_test_feature_target_ts(data_df=boston_df)

print(f"tr_ftr_ts shape:{tr_ftr_ts.shape} tr_tgt_ts shape:{tr_tgt_ts.shape}")
print(f"test_ftr_ts shape:{test_ftr_ts.shape} test_tgt_ts shape: {test_tgt_ts.shape}")

In [ ]:
# 학습 feature와 target으로 Stochastic Gradient Descent 수행. 
w1, w2, bias = st_gradient_descent(tr_ftr_ts, tr_tgt_ts, iter_epochs=5000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
# 테스트 데이터에서 예측 수행 및 결과를 DataFrame으로 생성. 
test_predicted_ts = test_ftr_ts[:, 0]*w1 + test_ftr_ts[:, 1]*w2 + bias

boston_test_df = pd.DataFrame({
    'RM': test_features[:, 0],
    'LSTAT': test_ftr_ts[:, 1],
    'PRICE': test_tgt_ts,
    'PREDICTED_PRICE_SGD': test_predicted_ts.cpu().numpy()
})

boston_test_df.head(20)

In [ ]:
from sklearn.metrics import mean_squared_error

test_total_mse = mean_squared_error(boston_test_df['PRICE'], boston_test_df['PREDICTED_PRICE_SGD'])
print(test_total_mse)

### iteration시마다 일정한 batch 크기만큼의 데이터를 random하게 가져와서 GD를 수행하는 Mini-Batch GD 수행

In [ ]:
def get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):
    # 데이터 건수
    N = target_batch.shape[0]
    # 예측 값. 
    predicted_batch = w1 * rm_batch + w2 * lstat_batch + bias
    # 실제값과 예측값의 차이
    diff_batch = target_batch - predicted_batch 
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N) * learning_rate * (torch.matmul(rm_batch, diff_batch))
    w2_update = -(2/N) * learning_rate * (torch.matmul(lstat_batch, diff_batch))
    bias_update = -(2/N) * learning_rate * torch.sum(diff_batch)
    
    # weight와 bias가 update되어야 할 값 반환. 
    return bias_update, w1_update, w2_update

In [ ]:
batch_indexes = torch.randint(0, 300, size=(30,))
print(batch_indexes)

tr_ftr_ts[batch_indexes, 0]

In [ ]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음. 
def batch_random_gradient_descent(features, target, iter_epochs=5000, batch_size=30, verbose=True):
    # random seed 값 설정. 
    torch.manual_seed(2025)
    # w1, w2는 1차원 tensor로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 tensor로 변환하되 초기 값은 1로 설정.
    w1 = torch.zeros(1, dtype=torch.float32)
    w2 = torch.zeros(1, dtype=torch.float32)
    bias = torch.ones(1, dtype=torch.float32)
    print('최초 w1, w2, bias:', w1.item(), w2.item(), bias.item())
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 tensor 형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(1, iter_epochs+1):
        # batch_size 갯수만큼 데이터를 임의로 선택. 
        batch_indexes = torch.randint(0, target.shape[0], size=(batch_size,))
        rm_batch = rm[batch_indexes]
        lstat_batch = lstat[batch_indexes]
        target_batch = target[batch_indexes]
        # Batch GD 기반으로 Weight/Bias의 Update를 구함. 
        bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, 
                                                                           rm_batch, lstat_batch, 
                                                                           target_batch, learning_rate)
        
        # Batch GD로 구한 weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose: # 100회 iteration 시마다 출력
            if i % 100 == 0:
                print(f'Epoch: {i}/{iter_epochs}')
                # Loss는 전체 학습 데이터 기반으로 구해야 함. 아래는 전체 학습 feature 기반의 예측 및 loss임.  
                predicted = w1 * rm + w2*lstat + bias
                diff = target - predicted
                loss = torch.mean(diff ** 2)
                print(f'w1: {w1.item()}, w2: {w2.item()}, bias: {bias.item()}, loss: {loss.item()}')
        
    return w1, w2, bias

In [ ]:
tr_ftr_ts, tr_tgt_ts, test_ftr_ts, test_tgt_ts = get_scaled_train_test_feature_target_ts(data_df=boston_df)

# 학습 feature와 target으로 Stochastic Gradient Descent 수행. 
w1, w2, bias = batch_random_gradient_descent(tr_ftr_ts, tr_tgt_ts, iter_epochs=5000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
# 테스트 데이터에서 예측 수행 및 결과를 DataFrame으로 생성. 
test_predicted_ts = test_ftr_ts[:, 0]*w1 + test_ftr_ts[:, 1]*w2 + bias

boston_test_df = pd.DataFrame({
    'RM': test_features[:, 0],
    'LSTAT': test_ftr_ts[:, 1],
    'PRICE': test_tgt_ts,
    'PREDICTED_PRICE_RANDOM_BATCH': test_predicted_ts.cpu().numpy()
})

test_total_mse = mean_squared_error(boston_test_df['PRICE'], boston_test_df['PREDICTED_PRICE_RANDOM_BATCH'])
print("test 데이터 세트의 MSE:", test_total_mse)

boston_test_df.head(20)

### iteration 시에 순차적으로 일정한 batch 크기만큼의 데이터를 전체 학습데이터에 걸쳐서 가져오는 Mini-Batch GD 수행

In [ ]:
for batch_step in range(0, 506, 30):
    print(batch_step)

In [ ]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음. 
def batch_gradient_descent(features, target, epochs=300, batch_size=30, verbose=True):
    # random seed 값 설정. 
    torch.manual_seed(2025)
    # w1, w2는 1차원 tensor로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 tensor로 변환하되 초기 값은 1로 설정.
    w1 = torch.zeros(1, dtype=torch.float32)
    w2 = torch.zeros(1, dtype=torch.float32)
    bias = torch.ones(1, dtype=torch.float32)
    print('최초 w1, w2, bias:', w1.item(), w2.item(), bias.item())
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(1, epochs+1):
        # batch_size 만큼 데이터를 가져와서 weight/bias update를 수행하는 로직을 전체 데이터 건수만큼 반복
        for batch_step in range(0, target.shape[0], batch_size):
            # batch_size만큼 순차적인 데이터를 가져옴. 
            rm_batch = rm[batch_step:batch_step + batch_size]
            lstat_batch = lstat[batch_step:batch_step + batch_size]
            target_batch = target[batch_step:batch_step + batch_size]
        
            bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, 
                                                                               rm_batch, lstat_batch, target_batch, 
                                                                               learning_rate)
            # Batch GD로 구한 weight/bias의 update 적용. 
            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update
        
        if verbose:
            print(f'Epoch: {i}/{epochs}')
            # Loss는 전체 학습 데이터 기반으로 구해야 함. 아래는 전체 학습 feature 기반의 예측 및 loss임.  
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            loss = torch.mean(diff ** 2)
            print(f'w1: {w1.item()}, w2: {w2.item()}, bias: {bias.item()}, loss: {loss.item()}')
        
    return w1, w2, bias

In [ ]:
tr_ftr_ts, tr_tgt_ts, test_ftr_ts, test_tgt_ts = get_scaled_train_test_feature_target_ts(data_df=boston_df)

# 학습 feature와 target으로 Mini Batch Gradient Descent 수행. 
w1, w2, bias = batch_gradient_descent(tr_ftr_ts, tr_tgt_ts, epochs=300, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
# 테스트 데이터에서 예측 수행 및 결과를 DataFrame으로 생성. 
test_predicted_ts = test_ftr_ts[:, 0]*w1 + test_ftr_ts[:, 1]*w2 + bias

boston_test_df = pd.DataFrame({
    'RM': test_features[:, 0],
    'LSTAT': test_ftr_ts[:, 1],
    'PRICE': test_tgt_ts,
    'PREDICTED_PRICE_BATCH': test_predicted_ts.cpu().numpy()
})

test_total_mse = mean_squared_error(boston_test_df['PRICE'], boston_test_df['PREDICTED_PRICE_BATCH'])
print("test 데이터 세트의 MSE:", test_total_mse)

boston_test_df.head(20)

### Pytorch를 이용하여 Simple Regression 모델 구축하기
* Pytorch 모델은 torch.nn.Module 클래스를 상속하여 생성함
* nn.Parameter()는 학습 파라미터(Learnable Parameter) tensor를 생성
* Pytorch의 train 로직은 model의 출력값(feed forward)을 오차 역전파(Backpropagation)로 weight 값 update 수행
* 손실 함수는 nn.MSELoss()로, Optimizer는 Adam 생성하고 모델 학습 수행.

In [ ]:
list(simple_model_01.parameters())

In [ ]:
import torch
import torch.nn as nn

class SimpleRegression_01(nn.Module):
    def __init__(self):
        super().__init__()
        #w
        self.weights = nn.Parameter(data=torch.zeros(size=(2, ), dtype=torch.float32),
                                   requires_grad=True)
        self.bias = nn.Parameter(data=torch.ones(size=(1,), dtype=torch.float32))
        
    def forward(self, x):
        return torch.matmul(self.weights, x.t()) + self.bias # w1*x1 + w2*x2 + b

simple_model_01 = SimpleRegression_01()
# simple_model의 학습 파라미터 출력(Learnable Parameter)
print(list(simple_model_01.parameters()))

In [ ]:
def get_scaled_train_test_feature_target_ts_01(data_df):
    # RM, LSTAT Feature에 Scaling 적용
    scaler = MinMaxScaler()
    scaled_features_np = scaler.fit_transform(data_df[['RM', 'LSTAT']])
    # 학습 feature, 테스트 feature, 학습 target, test_target으로 분리. 
    tr_features, test_features, tr_target, test_target = train_test_split(scaled_features_np, 
                                                                          data_df['PRICE'].values, 
                                                                          test_size=0.3, random_state=2025)
    # 학습 feature와 target을 tensor로 변환. dtype=torch.float32로 수정
    tr_ftr_ts = torch.tensor(tr_features, dtype=torch.float32)
    tr_tgt_ts = torch.tensor(tr_target, dtype=torch.float32)
    test_ftr_ts = torch.tensor(test_features, dtype=torch.float32)
    test_tgt_ts = torch.tensor(test_target, dtype=torch.float32)
    
    return tr_ftr_ts, tr_tgt_ts, test_ftr_ts, test_tgt_ts

In [ ]:
tr_ftr_ts, tr_tgt_ts, test_ftr_ts, test_tgt_ts = get_scaled_train_test_feature_target_ts_01(data_df=boston_df)
print(tr_ftr_ts.dtype, tr_tgt_ts.dtype, test_ftr_ts.dtype, test_tgt_ts.dtype)


In [ ]:
# MSELoss 생성. 
loss_fn = nn.MSELoss(reduction='mean')
# optimizer는 Adam으로, 생성 시 인자로 model의 모든 parameter 값과 learning rate가 필요. 
optimizer = torch.optim.Adam(simple_model_01.parameters(), lr=0.01)

# train loop 수행. 
def train_loop(model, tr_ftr_ts, tr_tgt_ts, loss_fn, optimizer, epochs=300, batch_size=30, verbose=True):
    #model.train()
    for i in range(1, epochs+1):
    # batch_size 만큼 데이터를 가져와서 weight/bias update를 수행하는 로직을 전체 데이터 건수만큼 반복
        for batch_step in range(0, tr_tgt_ts.shape[0], batch_size):
            # batch_size만큼 순차적인 데이터를 가져옴.
            ftr_batch = tr_ftr_ts[batch_step:batch_step + batch_size]
            target_batch = tr_tgt_ts[batch_step:batch_step + batch_size]
            
            # forward pass
            output = model(ftr_batch).squeeze(-1)
            
            # mse loss 계산
            loss = loss_fn(output, target_batch)

            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if verbose:
                if batch_step == 330:
                    print(f'Epoch: {i}/{epochs}, batch step:{batch_step}, loss: {loss.item()}')

    return model

In [ ]:
simple_model_01 = SimpleRegression_01()
loss_fn = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(simple_model_01.parameters(), lr=0.01)

trained_model = train_loop(simple_model_01, tr_ftr_ts, tr_tgt_ts, loss_fn, optimizer, 
                         epochs=300, batch_size=30, verbose=True) 

In [ ]:
from sklearn.metrics import mean_squared_error

test_predicted_ts = trained_model(test_ftr_ts)
print(test_predicted_ts.requires_grad, test_predicted_ts.shape)

boston_test_df = pd.DataFrame({
    'RM': test_features[:, 0],
    'LSTAT': test_ftr_ts[:, 1],
    'PRICE': test_tgt_ts,
    'PREDICTED': test_predicted_ts.squeeze(-1).detach().numpy()
})

test_total_mse = mean_squared_error(boston_test_df['PRICE'], boston_test_df['PREDICTED'])
print("test 데이터 세트의 MSE:", test_total_mse)

boston_test_df.head(20)

In [ ]:
import torch
import torch.nn as nn

class SimpleRegression_02(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features=2, out_features=1, bias=True)

    def forward(self, x):
        return self.linear(x)

simple_model_02 = SimpleRegression_02()
# simple_model의 학습 파라미터 출력(Learnable Parameter)
print(list(simple_model_02.parameters()))
print(simple_model_02)

In [ ]:
loss_fn = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(simple_model_02.parameters(), lr=0.01)
tr_ftr_ts, tr_tgt_ts, test_ftr_ts, test_tgt_ts = get_scaled_train_test_feature_target_ts_01(data_df=boston_df)

trained_model = train_loop(simple_model_02, tr_ftr_ts, tr_tgt_ts, loss_fn, optimizer, 
                         epochs=300, batch_size=30, verbose=True)

In [ ]:
from sklearn.metrics import mean_squared_error

test_predicted_ts = trained_model(test_ftr_ts)
print(test_predicted_ts.requires_grad, test_predicted_ts.shape)

boston_test_df = pd.DataFrame({
    'RM': test_features[:, 0],
    'LSTAT': test_ftr_ts[:, 1],
    'PRICE': test_tgt_ts,
    'PREDICTED': test_predicted_ts.squeeze(-1).detach().numpy()
})

test_total_mse = mean_squared_error(boston_test_df['PRICE'], boston_test_df['PREDICTED'])
print("test 데이터 세트의 MSE:", test_total_mse)

boston_test_df.head(20)

### 활성화 함수(Activation Function)
* function 기반으로 또는 Layer 기반으로 적용 가능
* relu()/ReLU()는 inplace=False(default)시 원본 입력 tensor를 복제하여 relu 적용. inplace=True시 원본 입력 tensor에 바로 relu를 적용하므로 원본 입력 tensor가 변환됨(메모리가 절감됨)
* softmax()/Softmax()의 경우는 dim=-1 을 보통 적용함

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

input_tensor = torch.tensor([0, 9.0, 10.0, -9.0, -10.0], dtype=torch.float32)

# sigmoid 적용
sigmoid_output = F.sigmoid(input_tensor)
# sigmoid_output = torch.sigmoid(input_tensor)

print('sigmoid_output:', sigmoid_output)

In [ ]:
# Sigmoid Layer 적용
sigmoid_layer = nn.Sigmoid()
sigmoid_output = sigmoid_layer(input_tensor)
print('sigmoid_output_02:', sigmoid_output)

In [ ]:
# relu() 함수 적용
input_tensor = torch.tensor([0, 9.0, 10.0, -9.0, -10.0], dtype=torch.float32)
# relu function 적용
# inplace=False시 입력 원본 tensor는 그대로 유지한 채 입력 tensor를 복사하여 relu함수 적용하여 반환
# inplace=True시 입력 원본 tensor에 바로 relu함수 적용하여 반환. 입력 tensor가 바로 relu 변환됨. 메모리 절감 
relu_output = F.relu(input_tensor, inplace=False) # inplace=True
print('relu_output:', relu_output)
# inplace=False시 input_tensor 값은 변환 없음. inplace=True시 input_tensor 값이 변화됨. 
print('input_tensor:', input_tensor)

In [ ]:
# ReLU Layer 적용
input_tensor = torch.tensor([0, 9.0, 10.0, -9.0, -10.0], dtype=torch.float32)
relu_layer = nn.ReLU() #inplace=True
relu_output = relu_layer(input_tensor)
print('relu_output:', relu_output)
print('input_tensor:', input_tensor)

In [ ]:
# softmax 함수 적용
input_tensor = torch.tensor([[1.0, -1.0, 0.5], 
                             [0.5, 0.5, 1.5]], dtype=torch.float32)
print('input_tensor shape:', input_tensor.shape)
softmax_output = F.softmax(input_tensor, dim=-1)
#softmax_output = torch.softmax(input_tensor, dim=-1)
print('softmax_output:', softmax_output)

#### Custom Model에서 Activation Function 사용

In [ ]:
import torch
import torch.nn as nn
#from torch import nn

# Custom Model 생성. 
class LinearModel(nn.Module):
    def __init__(self):
        # 반드시 super()를 호출. 
        super().__init__()
        #Linear Layer와 ReLU Layer 생성. 
        self.linear_01 = nn.Linear(in_features=10, out_features=5)
        self.relu_01 = nn.ReLU()
        self.linear_02 = nn.Linear(in_features=5, out_features=3)
        
    # 순방향 전파(Pass Forward) 기술.
    def forward(self, x):
        x = self.linear_01(x)
        x = self.relu_01(x)
        output = self.linear_02(x)
        return output

In [ ]:
#임의의 입력 tensor 생성. 
input_tensor = torch.randn(size=(4, 10))
print(input_tensor)

linear_model = LinearModel()

# LinearModel 객체는 Callable Object이므로 LinearModel 객체에 함수 호출과 유사한 형태로 입력 인자 전달하여 forward()메소드 호출. 
output_tensor = linear_model(input_tensor)
print(output_tensor)

In [ ]:
softmax_output = F.softmax(output_tensor, dim=-1)
#softmax_output = torch.softmax(input_tensor, dim=-1)
print('softmax_output:', softmax_output)
print('predicted class:', softmax_output.argmax(dim=-1))